In [1]:
import copy
import json
import os
import numpy
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union

from openai import BadRequestError

import autogen
from autogen import UserProxyAgent, AssistantAgent, ConversableAgent
from autogen import config_list_from_json
from autogen.agentchat import Agent
from autogen.agentchat.contrib.agent_optimizer import AgentOptimizer
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent
from autogen.code_utils import extract_code
from autogen.math_utils import get_answer

## Data

In [2]:
def read_file(url):
    file = open(url, "r")
    data = file.read().split('\n')
    return data

In [3]:
phoMT_dev_en = read_file("data/PhoMT/detokenization/dev/dev.en")
phoMT_dev_vi = read_file("data/PhoMT/detokenization/dev/dev.vi")
phoMT_test_en = read_file("data/PhoMT/detokenization/test/test.en")
phoMT_test_vi = read_file("data/PhoMT/detokenization/test/test.vi")
phoMT_train_en = read_file("data/PhoMT/detokenization/train/train.en")
phoMT_train_vi = read_file("data/PhoMT/detokenization/train/train.vi")

In [4]:
phoMT_dev_en[0] = phoMT_dev_en[0][1:]

In [5]:
phoMT_dev_envi = [];
for index in range(len(phoMT_dev_en)):
    phoMT_dev_envi.append({"question":phoMT_dev_en[index], "answer": phoMT_dev_vi[index]})
phoMT_test_envi = [];
for index in range(len(phoMT_test_en)):
    phoMT_test_envi.append({"question": phoMT_test_en[index], "answer": phoMT_test_vi[index]})
phoMT_train_envi = [];
for index in range(len(phoMT_train_en)):
    phoMT_train_envi.append({"question": phoMT_train_en[index], "answer": phoMT_train_vi[index]})
# phoMT_dev_envi

In [6]:
phoMT_dev_envi[0]["question"]

'Hurricane Dorian, one of the most powerful storms ever recorded in the Atlantic Ocean, made landfall as a Category 5 storm on Great Abaco Island in the northern Bahamas on Sunday morning, September 1, 2019.'

## Benchmark

### COMET

In [7]:
from comet import download_model, load_from_checkpoint

# Load the model checkpoint:
model = load_from_checkpoint('./XCOMET-XL/checkpoints/model.ckpt')

/home/kaylous/workspace/ics/llms/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Encoder model frozen.
/home/kaylous/workspace/ics/llms/.venv/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [8]:
def get_score_comet(src, ans, res):
    data = [
        {
            "src": src,
            "mt" : res,
            "ref": ans
        }
    ]
    return model.predict(data, batch_size=8, gpus=1).system_score

### SacreBleu

In [9]:
from sacrebleu.metrics import BLEU, CHRF, TER
bleu = BLEU()

def get_score(src, ans, res):
     return bleu.corpus_score([ans], [[res]]).score

## Agent init

### Agents declarations

In [10]:
llm_config = {
    "config_list": [
        {
            "model": "llama3",
            "base_url": "http://localhost:11434/v1",
            "api_key": "ollama",
        }
    ]
}

# user_proxy = autogen.UserProxyAgent(
#     name="Userproxyagent",
#     human_input_mode="NEVER",
#     code_execution_config={"work_dir": "_output", "use_docker": False},
# )

### Custom UserProxyAgent

In [11]:
class JudgeProxyAgent(UserProxyAgent):
    MAX_CONSECUTIVE_AUTO_REPLY = 10
    DEFAULT_REPLY_TEMPLATE = "Generate a response more closely resembling the style, detail, and tone of the provided answer. Focus on specifying key elements to capture the nuances of this answer effectively. The answer: "
    PROMPTS = """Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    """

    def __init__(
        self,
        name: Optional[str] = "JudgeChatAgent",
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "NEVER",
        **kwargs,
    ):
        super().__init__(
            name=name,
            human_input_mode=human_input_mode,
            **kwargs,
        )
        self.register_reply(
            trigger=ConversableAgent, reply_func=JudgeProxyAgent._check_final_result, position=0
        )
        self.max_function_call_trial = 3
        self.query = None
        self._answer = None
        self.is_correct = None
        self.previous = None
        # self.history = []

    def initiate_chat(
        self,
        recipient,
        history: List[Dict] = [],
        answer=None,
        previous=None,
        silent: Optional[bool] = False,
        max_turns = 3,
        **context,
    ):
        self.query = context["message"]
        self._answer = answer
        self.previous = previous
        
        self.is_correct = None
        self.max_function_call_trial = max_turns
        recipient.history = history
        print("Recipient.history: " + str(len(recipient.history)))
        
        self._prepare_chat(recipient, True)
        
        # for msg in history:
        #     self._append_oai_message(
        #         message=msg['content'],
        #         role=msg['role'],
        #         conversation_id = recipient if msg['role'] == "user" else self
        #     )
        
        error_message = None
        try:
            prompt = self.PROMPTS + context['message']
            self.send(prompt, recipient, silent=silent)
        except BadRequestError as e:
            error_message = str(e)
            self.is_correct = 0
            print("error information: {}".format(error_message))

        recipient.reset()
        self.is_correct = copy.deepcopy(self.is_correct)
        result = self.is_correct
        self._reset()
        return result

    def receive(
        self,
        message: Union[Dict, str],
        sender: Agent,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        self._process_received_message(message, sender, silent)
        if request_reply is False or request_reply is None and self.reply_at_receive[sender] is False:
            return

        self.is_correct = self.chat_messages[sender][-1].get("content")
        if self._answer is not None:
            if (get_score(self.query, self.is_correct, self._answer) >= 90):
                return

        self.max_function_call_trial = self.max_function_call_trial - 1
        if (self.max_function_call_trial <= 0):
            self.max_function_call_trial = 0
            return

        # reply = f"Using the original sentence: {self.query}, provide guidance to improve the quality of {self.is_correct}. Focus on enhancing accuracy, tone, fluency, and contextual appropriateness."
        reply = f"Translate the sentence: {self.query} into Vietnamese, ensuring that when the translation is appended to the previous translated paragraph: {self.previous}, the entire paragraph remains coherent, meaningful, and contextually appropriate. Respond with only the translated sentence, without any additional commentary or explanation."
        if self._answer is not None:
            reply = f'Analyze the original sentence: {self.query}, the expected Vietnamese translation: {self._answer}, and the generated translation: {self.is_correct}. Identify the differences between {self.is_correct} and {self._answer}, and provide guidance to improve the translation so it aligns more closely with {self._answer}. Focus on preserving meaning, tone, style, and naturalness in Vietnamese while addressing any discrepancies.'
        # if self._answer is not None:
        #     reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
        self.send(reply, sender, silent=silent)

    def _check_final_result(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[Agent] = None,
        config: Optional[Any] = None,
    ):
        messages = messages[-1]
        if isinstance(messages, dict):
            messages = messages.get("content")
            if messages is None:
                return False, None
            if (messages.find("\n") >= 0):
                print("Response longer than expected?\n" + messages)
                # messages = messages.split("\n")[0]

        self.is_correct = messages
        temp_score = get_score(self.query, messages, self._answer)
        print("Score: " + str(temp_score))
        if (temp_score >= 90):
            return True, "The result is passable. Please reply me with the same answer as before."
        return False, None

    def _reset(self):
        # super()._reset()
        self.max_function_call_trial = 0
        self.is_correct = None
        self.query = None
        self._answer = None
        self.previous = None

In [12]:
class PromptAssistant(AssistantAgent):
    PROMPTS = """Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    """
    TRIMMING_PROMPT = "Respond with only the improved translation, without any additional explanations or commentary."
    def __init__(
        self,
        after: Agent,
        before: Agent = None,
        name: Optional[str] = "PromptAssistantAgent",
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "NEVER",
        **kwargs,
    ):
        super().__init__(
            name=name,
            human_input_mode=human_input_mode,
            **kwargs,
        )
        self.before = before
        self.after = after
        self.history = []
        self.is_start = False
        self.last_msg = "NaH"

    def initiate_chat(
        self,
        recipient,
        history: List[Dict] = None,
        silent: Optional[bool] = False,
        max_turns = 3,
        **kwargs
    ):
        self.is_start = True
        self._prepare_chat(recipient, True)
        for msg in history:
            self._append_oai_message(
                message=msg['content'],
                role=msg['role'],
                conversation_id = recipient
            )
        print(len(self.chat_messages_for_summary(recipient)))
        error_message = None
        try:
            prompt = self.PROMPTS + kwargs['message']
            self.send(prompt, recipient, silent=silent)
        except BadRequestError as e:
            error_message = str(e)
            print("error information: {}".format(error_message))
        recipient.reset()
        self.is_start = False
        print(self.chat_messages_for_summary(recipient))
        return self.chat_messages_for_summary(recipient)[-1]['content']

    def receive(
        self,
        message: Union[Dict, str],
        sender: Agent,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        self._process_received_message(message, sender, silent)
        if request_reply is False or request_reply is None and self.reply_at_receive[sender] is False:
            return
        if self.is_start:
            # reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
            # if reply is not None:
            #     self.send(reply, sender, silent=silent)
            return
        if (sender == self.before):
            reply = sender.chat_messages_for_summary(self)[0]['content']
            if len(sender.chat_messages_for_summary(self)) > 1:
                state, res = self.generate_oai_reply(messages=self.chat_messages[sender], sender=sender)
                if state:
                    reply = res
                # reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
            else:
                print("Self.history: " + str(len(self.history)))
                for msg in self.history:
                    self._append_oai_message(
                        message=msg['content'],
                        role=msg['role'],
                        conversation_id = self.after
                    )
            if reply is not None:
                reply = reply + "\n\n" + self.TRIMMING_PROMPT
                self.last_msg = reply
                print(self.last_msg)
                self.send(reply, self.after, silent=silent, request_reply=True)
                if (self._oai_messages[self.after][-1]['content'].find('Note') != -1):
                    temp = self._oai_messages[self.after][-1]['content']
                    self._oai_messages[self.after][-1]['content'] = temp[:temp.find('Note')].strip()
                # print(self.chat_messages_for_summary(self.after))
                self.history = self.chat_messages_for_summary(self.after)
                self.send(self.chat_messages_for_summary(self.after)[-1]['content'], self.before, silent=silent)


In [13]:
LLM = autogen.AssistantAgent(
    name="LLM",
    system_message="You are a helpful assistant",
    code_execution_config=False,
    llm_config=llm_config,
)

Judge = JudgeProxyAgent(
    name="Judge",
    system_message="You are an advisor",
    code_execution_config=False,
)

PromptGenerator = PromptAssistant(
    before=Judge,
    after=LLM,
    name="PromptGenerator",
    system_message="You are a prompt engineer",
    human_input_mode = "NEVER",
    code_execution_config=False,
    llm_config=llm_config,
)

In [14]:
result = Judge.initiate_chat(
    recipient = PromptGenerator,
    max_turns = 2,
    message = phoMT_dev_envi[4]["question"],
    # answer = phoMT_dev_envi[4]["answer"]
)

Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    At this time, there have been no reported injuries among the 46 publishers in the two congregations on Great Abaco Island.

--------------------------------------------------------------------------------
Self.history: 0
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not inclu

In [15]:
PromptGenerator.last_msg
# PromptGenerator.history
    # print(Judge.chat_messages(PromptGenerator)[x])

'Đến thời điểm này, không có trường hợp thương tích được báo cáo trong số 46 ấn phẩm tại hai hội Thánh trên đảo Abaco lớn.\n\nRespond with only the improved translation, without any additional explanations or commentary.'

In [16]:
# PromptGenerator.chat_messages_for_summary(LLM)

In [17]:
# print(result)

In [18]:
# judge_result = "Tới thời điểm hiện nay, không có tin tức về người bị thương trong 46 Nhà xuất bản thuộc hai giáo hội trên đảo Abaco lớn."
# get_score(phoMT_dev_envi[4]["question"], result, phoMT_dev_envi[4]["answer"])

In [19]:
# result = PromptGenerator.initiate_chat(
#     recipient = LLM,
#     max_turns = 2,
#     message = phoMT_dev_envi[4]["question"],
#     history = PromptGenerator.history,
#     clear_history = False
# )

In [20]:
def score_translate(message, answer, turns = 1, history = None):
    if history is None:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            answer = answer,
        )
    else:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            answer = answer,
            history = history
        )
    return get_score(message, str(result), answer) / 100.0

def score_translate_test(message, answer, turns = 1, history = None, context = None):
    if history is None:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            previous = context
        )
    else:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            history = history,
            previous = context
        )
    # result = PromptGenerator.initiate_chat(
    #     recipient = LLM,
    #     max_turns = turns + 1,
    #     message = message,
    #     history = history,
    #     clear_history = False
    # )
    # return result
    return get_score(message, str(result), answer) / 100.0

def score_translate_comet(message, answer, turns = 1, history = None):
    if history is None:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            answer = answer,
        )
    else:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            answer = answer,
            history = history
        )
    return get_score_comet(message, str(result), answer)

def score_translate_comet_test(message, answer, turns = 1, history = None, context = None):
    if history is None:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            previous = context
        )
    else:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            history = history,
            previous = context
        )
    return get_score_comet(message, str(result), answer)

## Improve

In [21]:
len(phoMT_test_envi)

19152

In [22]:
train_num = 200
test_num = 50

### Before:

In [23]:
# score_dev = []
# for i in range(10):
#     x = phoMT_dev_envi[i]
#     score_dev.append(score_translate(x['question'], x['answer'], 0))
score_test = []
for i in range(test_num):
    x = phoMT_test_envi[train_num + i]
    score_test.append(score_translate(
        message= x['question'],
        answer= x['answer'],
        turns= 0
    ))

Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Additionally, Moscow authorities were guilty of violating the European Convention on Human Rights, to which Russia is a State Party.

--------------------------------------------------------------------------------
Self.history: 0
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do

In [24]:
round(numpy.average(score_test), 2)

0.3

In [25]:
score_test

[0.141598876330713,
 0.11191021613657742,
 0.2590893643584191,
 0.07434360163166209,
 0.07452895202847676,
 0.27075392202374643,
 0.24783668394234723,
 0.2444483415737007,
 0.6028817681965138,
 0.43821784388200513,
 0.616794087239726,
 0.23423016820418144,
 0.28164974576164165,
 0.4539996117475736,
 0.3559799811720743,
 0.4191355943282005,
 0.41578384349321884,
 0.12355567759787611,
 0.26539789846276113,
 0.5366411241731205,
 0.05665374508695042,
 0.23423729036817917,
 0.2805155032016295,
 0.26512298021756175,
 0.09906944862624854,
 0.6153307664945966,
 0.3173772916054519,
 0.3448035019916339,
 0.08597625399625076,
 0.3025371225363357,
 0.09107059268540424,
 0.3119875475467477,
 0.2639627717091577,
 0.4176067181205533,
 0.22427511279116985,
 0.48806767957875097,
 0.26230609318615344,
 0.2820059340102563,
 0.4202431608906194,
 0.3040240343804179,
 0.24452946438904896,
 0.31669349232191585,
 0.11281745647221446,
 0.5328937378954782,
 0.098423306167077,
 0.37034371526508314,
 0.1735802430

In [26]:
score_test = []
for i in range(test_num):
    x = phoMT_test_envi[train_num + i]
    score_test.append(score_translate_comet(
        message= x['question'],
        answer= x['answer'],
        turns= 0
    ))

Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Additionally, Moscow authorities were guilty of violating the European Convention on Human Rights, to which Russia is a State Party.

--------------------------------------------------------------------------------
Self.history: 0
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.85s/it]


Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    In 2020, as in 2010, Russia's systemic campaign against our brothers is not going unnoticed.

--------------------------------------------------------------------------------
Self.history: 0
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introduct

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.87s/it]

Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    'For Jehovah's Witnesses in Russia, practicing their faith means risking their freedom,' said Rachel Denber, deputy Europe and Central Asia director for Human Rights Watch.

--------------------------------------------------------------------------------
Self.history: 0
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Re

...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.92s/it]

Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    After the accident had occurred, the organizers of the marathon suspended the competition and used vehicles to transport runners back to the starting point.

--------------------------------------------------------------------------------
Self.history: 0
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with 


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.30s/it]

Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Over 6,200 local and foreign runners are joining the Dalat Ultra Trail 2020.

--------------------------------------------------------------------------------
Self.history: 0
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up 


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.21s/it]


In [27]:
round(numpy.average(score_test), 2)

0.91

In [28]:
score_test

[0.8741694688796997,
 0.8285999298095703,
 0.6615022420883179,
 0.9638828039169312,
 0.7150874733924866,
 0.8696470856666565,
 0.9680848121643066,
 0.9305007457733154,
 0.9754449129104614,
 0.9750561714172363,
 1.0,
 0.9066888689994812,
 0.8827180862426758,
 0.7525320053100586,
 0.9968845844268799,
 1.0,
 0.9949772357940674,
 0.8534387350082397,
 0.9660583138465881,
 0.9235293865203857,
 0.9335551261901855,
 0.9885571002960205,
 0.9822101593017578,
 0.9907200336456299,
 0.9069826006889343,
 0.9465160369873047,
 0.9475220441818237,
 0.9467480182647705,
 0.941399335861206,
 0.9437307119369507,
 0.8509376645088196,
 0.9827957153320312,
 0.9581100940704346,
 0.9888616800308228,
 0.993569016456604,
 0.9903988838195801,
 0.8917264342308044,
 0.5563387274742126,
 0.9747229814529419,
 0.9227638244628906,
 0.9456907510757446,
 0.9990181922912598,
 0.9856445789337158,
 0.9329491853713989,
 0.9526363611221313,
 0.8780308365821838,
 0.6007892489433289,
 0.9158682227134705,
 0.7901568412780762,
 0.

In [29]:
len(phoMT_train_envi)

2978000

### During

In [30]:
EPOCH = 1
turns = 1
# optimizer_model = "gpt-4-1106-preview"
# optimizer = AgentOptimizer(max_actions_per_step=3, llm_config=llm_config, optimizer_model="llama3")
history_recorder = []
last_msg_s = []
for index in range(train_num):
    query = phoMT_test_envi[index]
    # is_correct = user_proxy.initiate_chat(assistant, answer=query["answer"], problem=query["question"])
    result = score_translate(query['question'], query['answer'], turns)
    history = PromptGenerator.history
    print(f"Test: {index}")
    history_recorder.extend(history)
    last_msg_s.append(PromptGenerator.last_msg)
    print(len(history_recorder))

Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Brother Albert Barnett and his wife, Sister Susan Barnett, from the West Congregation in Tuscaloosa, Alabama

--------------------------------------------------------------------------------
Self.history: 0
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explana

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 09:50:33] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Vương hậu nhạc pop của V - prince trong bản phim tài liệu soundtrack được xếp hạng âm nhạc toàn cầu.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Vương hậu nhạc pop của V - prince trong bản phim tài liệu soundtrack được xếp hạng âm nhạc toàn cầu.

--------------------------------------------------------------------------------
Judge (to PromptGenerator):

Analyze the original sentence: V - pop prince's documentary soundtrack among global music charts, the expected Vietnamese translation: Nhạc phim tài liệu của Hoàng tử V-pop lọt vào các bảng xếp hạng âm nhạc toàn cầu, and the generated translation: Vương hậu nhạc pop của V - prince trong bản phim tài liệu soundtrack được xếp hạng âm nhạc toàn cầu.. Identify the differences between Vương hậu nhạc pop của V - prince trong bản phim tài liệu soundtrack được xếp hạng âm nhạc toàn cầu. and Nhạc phim tài liệu của Hoàng tử V-pop lọt vào các bảng xếp hạ

...

Here's the analysis:

1. Differences between the two translations:
	* "Tóm lại" (Summary) in the generated translation vs. "Để kết luận" (To conclude) in the expected translation.
	* "Kết luận" (Conclude) in the generated translation is redundant, whereas it's not present in the expected translation.
	* "Ủy ban Tư pháp Moskva cùng với các Toà án Moskva" (The Moscow Justice Department, along with the Moscow courts) in the generated translation vs. "Sở Tư pháp thành phố Matxcova và Toà án Matxcova" (The Moscow Justice Department and the Moscow courts) in the expected translation.
	* "Khoán lại Đăng ký của Nhân Chứng Jehovah" (for refusing the reregistration of Jehovah's Witnesses) in the generated translation vs. "để từ chối cấp phép hoạt động trở lại cho Nhân Chメント Jehovah" (to refuse permission for Jehovah's Witnesses to operate again) in the expected translation.
	* "Tòa án đã phán xét về các quan chức Moskva" (The Court decried the Moscow authorities) in the generated translation vs.

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:26:59] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Để kết luận, Tòa án Nhân quyền Châu Âu đã kết luận rằng Sở Tư pháp thành phố Matxcova và Toà án Matxcova không có cơ sở pháp lý để từ chối cấp phép hoạt động trở lại cho Nhân Chệnh Jehovah. Toà án đã công khai chỉ trích chính quyền thành phố Matxcova, khẳng định rằng họ "không hành động trong thiện chí và bỏ qua nghĩa vụ trung lập và vô tư".

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Để kết luận, Tòa án Nhân quyền Châu Âu đã kết luận rằng Sở Tư pháp thành phố Matxcova và Toà án Matxcova không có cơ sở pháp lý để từ chối cấp phép hoạt động trở lại cho Nhân Chệnh Jehovah. Toà án đã công khai chỉ trích chính quyền thành phố Matxcova, khẳng định rằng họ "không hành động trong thiện chí và bỏ qua nghĩa vụ trung lập và vô tư".

--------------------------------------------------------------------------------
Test: 199
797


In [31]:
len(history_recorder)

797

### After

In [32]:
score_test = []
answers = []
for i in range(test_num):
    x = phoMT_test_envi[train_num+i]
    print("Test " + str(i) + ":")
    context = " ".join(answers)
    # if (len(answers)>5) context = answers[-5:].join(" ")
    score_test.append(score_translate_test(
        message = x['question'],
        answer = x['answer'],
        turns = 0,
        history = history_recorder,
        context = context
    ))

Test 0:
Recipient.history: 797
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Additionally, Moscow authorities were guilty of violating the European Convention on Human Rights, to which Russia is a State Party.

--------------------------------------------------------------------------------
Self.history: 797
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requeste

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:27:03] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Thêm vào đó, chính quyền thành phố Matxcova đã phạm phải vi phạm Hiệp định Nhân quyền Châu Âu, vốn là một hiệp ước mà Nga là một bên ký kết.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Thêm vào đó, chính quyền thành phố Matxcova đã phạm phải vi phạm Hiệp định Nhân quyền Châu Âu, vốn là một hiệp ước mà Nga là một bên ký kết.

--------------------------------------------------------------------------------
Test 1:
Recipient.history: 797
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
 

...

LLM (to PromptGenerator):

Sau tai nạn xảy ra, ban tổ chức cuộc chạy marathon đã tạm dừng cuộc thi và sử dụng các phương tiện để chuyển những người chạy về điểm xuất phát.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Sau tai nạn xảy ra, ban tổ chức cuộc chạy marathon đã tạm dừng cuộc thi và sử dụng các phương tiện để chuyển những người chạy về điểm xuất phát.

--------------------------------------------------------------------------------
Test 49:
Recipient.history: 797
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    Th

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:30:14] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Hơn 6,200 người chạy địa phương và nước ngoài tham gia cuộc chạyDalat Ultra Trail 2020.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Hơn 6,200 người chạy địa phương và nước ngoài tham gia cuộc chạyDalat Ultra Trail 2020.

--------------------------------------------------------------------------------


In [33]:
round(numpy.average(score_test), 2)

0.33

In [34]:
score_test

[0.24883462399716788,
 0.05682676011595584,
 0.013669512658275674,
 0.03306950185909273,
 0.06622923495388001,
 0.19522904561990737,
 0.421848315237853,
 0.22483948425730904,
 0.6028817681965138,
 0.36381467802469103,
 0.6392938829148984,
 0.3005180557366462,
 0.14557035150304856,
 0.7367565054628354,
 0.6262844962765475,
 0.1332358437599213,
 0.4275390339632901,
 0.5151777555161081,
 0.7976132786630805,
 0.40108897145389916,
 0.4323395762078184,
 0.5153978826718985,
 0.19209534151258667,
 0.363514666524577,
 0.16684648325595255,
 0.6381361864872335,
 0.2943848546995405,
 0.32240153143469596,
 0.201729540378002,
 0.29479280846982564,
 0.2677228663419846,
 0.35934029774604015,
 0.45227603829460317,
 0.4845393980644712,
 0.26642370598020887,
 0.42457427829511013,
 0.34506029095630353,
 0.12838391376934297,
 0.1721916742076807,
 0.13901391648593878,
 0.40589517638127054,
 0.10419462900243012,
 0.1165393923449734,
 0.4636133385196177,
 0.1892240568795936,
 0.2932436633690613,
 0.2286232839

In [35]:
score_test = []
answers = []
for i in range(test_num):
    x = phoMT_test_envi[train_num+i]
    print("Test " + str(i) + ":")
    context = " ".join(answers)
    # if (len(answers)>5) context = answers[-5:].join(" ")
    score_test.append(score_translate_test(
        message = x['question'],
        answer = x['answer'],
        turns = 1,
        history = history_recorder,
        context = context
    ))

Test 0:
Recipient.history: 797
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Additionally, Moscow authorities were guilty of violating the European Convention on Human Rights, to which Russia is a State Party.

--------------------------------------------------------------------------------
Self.history: 797
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requeste

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:30:18] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Thêm vào đó, các cơ quan Moscow được phát hiện vi phạm Công ước Nhân quyền châu Âu, nước Nga là một bên tham gia.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Thêm vào đó, các cơ quan Moscow được phát hiện vi phạm Công ước Nhân quyền châu Âu, nước Nga là một bên tham gia.

--------------------------------------------------------------------------------
Judge (to PromptGenerator):

Translate the sentence: Additionally, Moscow authorities were guilty of violating the European Convention on Human Rights, to which Russia is a State Party. into Vietnamese, ensuring that when the translation is appended to the previous translated paragraph: , the entire paragraph remains coherent, meaningful, and contextually appropriate. Respond with only the translated sentence, without any additional commentary or explanation.

--------------------------------------------------------------------------------


...

Hơn 6,200 người chạy bộ địa phương và nước ngoài tham gia cuộc đua Dalat Ultra Trail 2020.

Respond with only the improved translation, without any additional explanations or commentary.
PromptGenerator (to LLM):

Hơn 6,200 người chạy bộ địa phương và nước ngoài tham gia cuộc đua Dalat Ultra Trail 2020.

Respond with only the improved translation, without any additional explanations or commentary.

--------------------------------------------------------------------------------


HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:38:51] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Hơn 6,200 người chạy bộ địa phương và người nước ngoài tham gia cuộc đua Dalat Ultra Trail 2020.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Hơn 6,200 người chạy bộ địa phương và người nước ngoài tham gia cuộc đua Dalat Ultra Trail 2020.

--------------------------------------------------------------------------------


In [36]:
round(numpy.average(score_test), 2)

0.24

In [37]:
score_test

[0.09495166149149357,
 0.10275474580767145,
 0.16249885760085053,
 0.06199916348755865,
 0.12916903695846407,
 0.103403665348308,
 0.19534557593516516,
 0.21024692077841564,
 0.12109013026441869,
 0.43821784388200513,
 0.2336023677844039,
 0.22320859453962552,
 0.13532043663194476,
 0.23287896954139942,
 0.5265753584723772,
 0.4191355943282005,
 0.3664919453964599,
 0.12738938828520563,
 0.4957412168254517,
 0.4076128826387552,
 0.1271959338344984,
 0.4815541843509281,
 0.23693055763743093,
 0.11047111196276083,
 0.1143955456656648,
 0.36683476735395104,
 0.26735519390703294,
 0.18694976491030255,
 0.15286714039797208,
 0.08980949266218706,
 0.09107059268540424,
 0.30814893260907833,
 0.3082355804637448,
 0.38135958308066087,
 0.22022141213630328,
 0.4212800597133499,
 0.2464973065155432,
 0.22580433890796045,
 0.3712973183915444,
 0.04025562633372525,
 0.12088736774295636,
 0.10117127007355416,
 0.13275872457704752,
 0.4063044680074321,
 0.11369306152454815,
 0.3493616226851873,
 0.35

In [38]:
score_test = []
answers = []
for i in range(test_num):
    x = phoMT_test_envi[train_num+i]
    print("Test " + str(i) + ":")
    context = " ".join(answers)
    # if (len(answers)>5) context = answers[-5:].join(" ")
    score_test.append(score_translate_comet_test(
        message = x['question'],
        answer = x['answer'],
        turns = 0,
        history = history_recorder,
        context = context
    ))

Test 0:
Recipient.history: 797
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Additionally, Moscow authorities were guilty of violating the European Convention on Human Rights, to which Russia is a State Party.

--------------------------------------------------------------------------------
Self.history: 797
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requeste

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:38:55] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Ngoài ra, các cơ quan chức năng Moskva cũng đã vi phạm Công ước Nhân quyền châu Âu, nước Nga là một bên tham gia Công ước này.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Ngoài ra, các cơ quan chức năng Moskva cũng đã vi phạm Công ước Nhân quyền châu Âu, nước Nga là một bên tham gia Công ước này.

--------------------------------------------------------------------------------


...


HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:46:51] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Hơn 6,200 người chạy bản địa và quốc tế đã tham gia đường đuaDalat Ultra Trail 2020.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Hơn 6,200 người chạy bản địa và quốc tế đã tham gia đường đuaDalat Ultra Trail 2020.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.59s/it]


In [39]:
round(numpy.average(score_test), 2)

0.92

In [40]:
score_test

[0.9097382426261902,
 0.8384610414505005,
 0.8736088275909424,
 0.9266725778579712,
 0.8814340829849243,
 0.8444397449493408,
 0.96491938829422,
 0.9054682850837708,
 0.955181896686554,
 0.9770528078079224,
 0.9871245622634888,
 0.9054643511772156,
 0.7692790031433105,
 0.8274586200714111,
 0.9947426319122314,
 0.9949395656585693,
 0.9961991310119629,
 0.8809948563575745,
 0.9076696634292603,
 0.9614629149436951,
 0.8718713521957397,
 0.9819519519805908,
 0.9822101593017578,
 0.9892492294311523,
 0.9460265636444092,
 0.9268882870674133,
 0.9394201636314392,
 0.9608500003814697,
 0.9306182861328125,
 0.9501844644546509,
 0.8143892288208008,
 0.9722409248352051,
 0.9482970833778381,
 0.9837360382080078,
 0.9927761554718018,
 0.9941641092300415,
 0.9347525835037231,
 0.6123239994049072,
 0.9947717189788818,
 0.9262309074401855,
 0.8083716630935669,
 0.9993678331375122,
 0.9878089427947998,
 0.8894113898277283,
 0.9359546899795532,
 0.8380188345909119,
 0.6019043922424316,
 0.9183264970779

In [ ]:
score_test = []
answers = []
for i in range(test_num):
    x = phoMT_test_envi[train_num+i]
    print("Test " + str(i) + ":")
    context = " ".join(answers)
    # if (len(answers)>5) context = answers[-5:].join(" ")
    score_test.append(score_translate_comet_test(
        message = x['question'],
        answer = x['answer'],
        turns = 1,
        history = history_recorder,
        context = context
    ))

Test 0:
Recipient.history: 797
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Additionally, Moscow authorities were guilty of violating the European Convention on Human Rights, to which Russia is a State Party.

--------------------------------------------------------------------------------
Self.history: 797
Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requeste

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:46:59] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to PromptGenerator):

Ngoài ra, các cơ quan chức năng Moskva còn được tìm thấy vi phạm Công ước Châu Âu về quyền con người, trong đó Nga là một bên tham gia.

--------------------------------------------------------------------------------
PromptGenerator (to Judge):

Ngoài ra, các cơ quan chức năng Moskva còn được tìm thấy vi phạm Công ước Châu Âu về quyền con người, trong đó Nga là một bên tham gia.

--------------------------------------------------------------------------------
Judge (to PromptGenerator):

Translate the sentence: Additionally, Moscow authorities were guilty of violating the European Convention on Human Rights, to which Russia is a State Party. into Vietnamese, ensuring that when the translation is appended to the previous translated paragraph: , the entire paragraph remains coherent, meaningful, and contextually appropriate. Respond with only the translated sentence, without any additional commentary or explanation.

-------------------------------------------

...

Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Tử vong đã vượt qua 2.700, chủ yếu tại Trung Quốc, sau là Iran với 15 trường hợp, Hàn Quốc với 11 và Ý với 7 trường hợp; Tử vong khác gồm những quốc gia khác.

Respond with only the improved translation, without any additional explanations or commentary.
PromptGenerator (to LLM):

Tử vong đã vượt qua 2.700, chủ yếu tại Trung Quốc, sau là Iran với 15 trường hợp, Hàn Quốc với 11 và Ý với 7 trường hợp; Tử vong khác gồm những quốc gia khác.

Respond with only the improved translation, without any additional explanations or commentary.

--------------------------------------------------------------------------------


HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-28 10:58:27] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [ ]:
round(numpy.average(score_test), 2)

In [ ]:
score_test